In [ ]:
class Strategy:
    def __init__(self, data) -> None:
        self.data = data
        self.indicator = None

data = [0,1,2,3,4,5]
strat = Strategy(data)
strat.__dict__.items()

In [ ]:
print(len.__name__)
print(len.__class__.__name__)

In [ ]:
import datetime as dt 
import yfinance as yf
import pandas as pd

num_days = 730
start=dt.date(2021, 1, 1)
start = max(start, dt.date.today() - dt.timedelta(days=num_days-1))
end = dt.date.today() 
print(start, end)
data = yf.download("SPY", interval="1h", start=start, end=end, prepost=True)
data = data.drop(['Adj Close'], axis=1)
data.index = list(map(lambda x: dt.datetime.strptime(str(x).replace(":",""), '%Y-%m-%d %H%M%S%z').replace(tzinfo=None), data.index))
data = data.rename_axis("Datetime").reset_index()
print(data)


In [ ]:
timestr = '2022-07-28 04:00:00'
output = dt.datetime.strptime(timestr, '%Y-%m-%d %H:%M:%S')
print(output)


In [ ]:
import bokeh.colors

# for c in dir(bokeh.colors.named):
    # print(c, c.__class__)
import bokeh.palettes
print(bokeh.palettes.Dark2_8)

# print(bokeh.colors.named.blue.__class__)
# print(all_colors)

# Top gainers aftermath

In [ ]:
from tradingview_screener import *
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

n_rows, df = (Query()
 .select('name', 'close', 'change', 'volume', 'relative_volume_10d_calc', 'market_cap_basic', 'float_shares_outstanding')
 .where(
     Column('market_cap_basic') >= 1_000_000,
     Column('relative_volume_10d_calc') > 1.2,
     Column('change') > 5,
     Column('volume') > 1E6,
     Column('close') > 2,
    )
 .order_by('change', ascending=False)
 .get_scanner_data())

df.volume = df.volume * 1E-6
df.market_cap_basic = df.market_cap_basic * 1E-6
df.float_shares_outstanding = df.float_shares_outstanding * 1E-6
df = df.rename(columns={
    "volume": "Volume (M)", 
    "market_cap_basic": "Market Cap (M)",
    "change": "Change %",
    "float_shares_outstanding": "Float (M)"
    })
df = df[df.ticker.str.contains("NASDAQ:") | df.ticker.str.contains("NYSE:")]
print(len(df))
display(df)


In [ ]:
import datetime as dt
from pandas.tseries.offsets import BDay

(dt.date(2024, 9, 1) - BDay(0)).date()

In [ ]:
import datetime as dt
from pandas.tseries.offsets import BDay

# all_data = {dt.date.today() : df.ticker.to_list()}
fake_date_range = [dt.date(2024, 9, i) for i in range(1, 30)]
fake_data = {fake_date_range[i//2] : [str(j) for j in range(i, i+3)] for i in range(0, len(fake_date_range)*2,2)}
print(fake_data)


observation_time = 1 # days after making top gainer list
for today_date in fake_date_range:
    stocks_to_scan = set()
    for time_delta in range(observation_time + 1):
        # lookback_date = (today_date - BDay(time_delta)).date()
        lookback_date = today_date - dt.timedelta(days=time_delta)
        print(lookback_date)
        if lookback_date in fake_data:
            stocks_to_scan.update(fake_data[lookback_date])
            print("updated:", fake_data[lookback_date])
    print(today_date, stocks_to_scan)
    print()
        

# Premarket

In [ ]:
from tradingview_screener import *
import pandas as pd

pd.options.display.float_format = '{:,.2f}'.format

n_rows, df = (
    Scanner.premarket_gainers
    .select('name', 'close', 'volume', 'premarket_change', 'premarket_volume', 'relative_volume_10d_calc', 'market_cap_basic', 'float_shares_outstanding')
    .where(
     Column('market_cap_basic') >= 1_000_000,
     Column('relative_volume_10d_calc') > 1.2,
     Column('premarket_change') > 5,
     Column('premarket_volume') > 1E6,
     Column('close') > 1
    )
    .get_scanner_data())


df.volume = df.volume * 1E-6
df.market_cap_basic = df.market_cap_basic * 1E-6
df.premarket_volume = df.premarket_volume * 1E-6
df.float_shares_outstanding = df.float_shares_outstanding * 1E-6
df = df.rename(columns={
    "volume": "Volume (M)", 
    "market_cap_basic": "Market Cap (M)",
    "premarket_change": "Premarket %",
    "premarket_change_abs": "Premarket $",
    "premarket_volume": "Premarket Volume (M)",
    "float_shares_outstanding": "Float (M)"
    })
print(n_rows)

display(df)

In [ ]:
import datetime as dt
if dt.date.weekday(dt.date.today()) < 5:
    pass
from download import *
from visualize import *

yf_df = update_from_yf("EBS", "1m", dt.datetime(2024,8,28), dt.datetime(2024,8,30))

# visualize(df)


In [ ]:
# pd.set_option('display.max_rows', 2500)
display(yf_df)

In [ ]:
import requests
import pandas as pd
import datetime as dt

url = 'https://api.polygon.io/v2/aggs/ticker/EBS/range/1/minute/2023-08-01/2023-08-30?adjusted=true&sort=asc&limit=50000&apiKey=*'
r = requests.get(url)
if not (r.json() and "results" in r.json() and "status" in r.json() and r.json()["status"] == "OK"):
    raise RuntimeError("Polygon JSON Error!")
results = r.json()['results']
# df = pd.DataFrame(columns=["Datetime", 'Open', 'High', 'Low', 'Close', 'Volume'])
df = pd.json_normalize(results)
df.t = pd.Series.apply(df.t, lambda x: dt.datetime.fromtimestamp(x*1E-3).astimezone(dt.timezone(dt.timedelta(hours=-4))).replace(tzinfo=None))


In [ ]:
print(r.json()['count'])

In [ ]:
display(df)

In [ ]:
market_open = pd.to_datetime('09:30:00').time()
market_close = pd.to_datetime('16:00:00').time()
df = df.rename(columns={"v": "Volume", "vw": "VWAP", "o": "Open", "c": "Close", "h":"High", "l":"Low", "t":"Datetime"})

# TODO: Clean up this code

pre_post_data = df.loc[(df['Datetime'].dt.time <= market_open) | (df['Datetime'].dt.time >= market_close)]

# TODO: Lookup not by exact value, but by the closest timestamp in yf_df after the query timestamp. 
for index, row in pre_post_data.iterrows():
    yf_row = yf_df[yf_df['Datetime'] == row.Datetime]
    assert len(yf_row) <= 1, "Multiple of the same timestamps found."
    yf_df.loc[yf_row.index, 'Volume'] = row.Volume
    

# pre_post_data["closest_timestamp"] = pd.merge_asof(pre_post_data, yf_df, on='Datetime', direction='forward')['Datetime']




In [31]:
import pandas_market_calendars as mcal
import pandas as pd
import datetime as dt

market_schedule = mcal.get_calendar('NYSE').schedule(start_date='2024-01-01', end_date='2024-12-31')
regular_business_days = market_schedule.loc[market_schedule['market_close'].dt.time >= pd.to_datetime('20:00:00').time()].index.to_list()

regular_business_days = [i.to_pydatetime().date() for i in regular_business_days]
print(regular_business_days)
print((dt.datetime.now().date() + dt.timedelta(days=1)) in regular_business_days)

[datetime.date(2024, 1, 2), datetime.date(2024, 1, 3), datetime.date(2024, 1, 4), datetime.date(2024, 1, 5), datetime.date(2024, 1, 8), datetime.date(2024, 1, 9), datetime.date(2024, 1, 10), datetime.date(2024, 1, 11), datetime.date(2024, 1, 12), datetime.date(2024, 1, 16), datetime.date(2024, 1, 17), datetime.date(2024, 1, 18), datetime.date(2024, 1, 19), datetime.date(2024, 1, 22), datetime.date(2024, 1, 23), datetime.date(2024, 1, 24), datetime.date(2024, 1, 25), datetime.date(2024, 1, 26), datetime.date(2024, 1, 29), datetime.date(2024, 1, 30), datetime.date(2024, 1, 31), datetime.date(2024, 2, 1), datetime.date(2024, 2, 2), datetime.date(2024, 2, 5), datetime.date(2024, 2, 6), datetime.date(2024, 2, 7), datetime.date(2024, 2, 8), datetime.date(2024, 2, 9), datetime.date(2024, 2, 12), datetime.date(2024, 2, 13), datetime.date(2024, 2, 14), datetime.date(2024, 2, 15), datetime.date(2024, 2, 16), datetime.date(2024, 2, 20), datetime.date(2024, 2, 21), datetime.date(2024, 2, 22), date